In [49]:
#import numpy as np
import pandas as pd
import csv

#import people data from csv
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1')

#import juniors data and convert to a list
import_juniors =  pd.read_csv('../custom data/juniors.csv')
juniors = import_juniors['playerid'].values.tolist()

#import birth region file
import_regions = pd.read_csv('../custom data/regions.csv')


In [45]:
def jr(i, n):
    if i in juniors:
        return f"{n} Jr."
    else:
        return n
    
def gen(x):
    if 1700 <= x <= 1882:
        return 'Pre-Generational'
    elif 1883 <= x <= 1900:
        return 'Lost Generation'
    elif 1901 <= x <= 1927:
        return 'Greatest Generation'
    elif 1928 <= x <= 1945:
        return 'Silent Generation'
    elif 1946 <= x <= 1964:
        return 'Baby Boomers'
    elif 1965 <= x <= 1980:
        return 'Gen X'
    elif 1981 <= x <= 1996:
        return 'Millenial'
    elif 1997 <= x <= 2012:
        return 'Gen Z'
    elif 2013 <= x <= 2025:
        return 'Gen Alpha'
    else:
        return 'Unknown'

In [53]:

#remove non-players, remove unneeded columns
people_df = import_people[~import_people['playerID'].str.endswith('99')].drop(['ID', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameGiven'], axis=1).copy()

#fix blanks and cast as integer where appropriate
people_df['birthYear'] = people_df['birthYear'].fillna(0).astype(int)
people_df['birthMonth'] = people_df['birthMonth'].fillna(0).astype(int)
people_df['birthDay'] = people_df['birthDay'].fillna(0).astype(int)
people_df['weight'] = people_df['weight'].fillna(0).astype(int)
people_df['height'] = people_df['height'].fillna(0).astype(int)
people_df['birthCountry'] = people_df['birthCountry'].fillna('Unknown').apply(lambda x: x if x != 'Ukriane' else 'Ukraine')
people_df['birthCity'] = people_df['birthCity'].fillna('Unknown')
people_df['birthState'] = people_df['birthState'].fillna('NA')
people_df['birthCity'] = people_df.apply(lambda x: f"{x['birthCity']}, {x['birthState']}", axis=1)
people_df['nameFirst'] = people_df['nameFirst'].fillna('Unknown')
people_df['bats'] = people_df['bats'].fillna('U')
people_df['throws'] = people_df['throws'].fillna('U')
people_df['bbrefID'] = people_df['bbrefID'].fillna('xxx1234')
people_df['debutDate'] = people_df['debut'].fillna('1800-01-01')
people_df['finalGame'] = people_df['finalGame'].fillna('1800-01-01')

#concatenate birth day
people_df['birthDate'] = people_df.apply(lambda x: f"{x['birthYear']}-{str(x['birthMonth']).zfill(2)}-{str(x['birthDay']).zfill(2)}", axis=1)

#juniorize and make the full name
people_df['nameLast'] = people_df.apply(lambda x: jr(x['playerID'], x['nameLast']), axis=1)
people_df['fullName'] = people_df.apply(lambda x: f"{x['nameFirst']} {x['nameLast']}", axis=1)

#determine player generation
people_df['generation'] = people_df['birthYear'].apply(lambda x: gen(x))

#determine birth region
people_df = people_df.merge(import_regions, how='left', left_on='birthCountry', right_on='country')


#TODO: Rename Enrique hernandez to Kike hernandez, Rowdy Tellez to Ryan Tellez, Covelli to coco crisp, George runie farmer to buck farmer
#TODO: research players who go by nicknames or shortened names
#TODO: apply birth region
#TODO: determine players primary team
#TODO: Calculate player salary -- do we really need to tho??
#TODO: determine player primary position and position group
#TODO: determine hof status
#TODO: sum player appearances
#TODO: calculate baseline data like ba, ip, inns on defence


#TODO: drop debut, birth year, birth date, birth month column
#TODO: order columns



print(people_df)

        playerID  birthYear  birthMonth  birthDay         birthCity  \
0      aardsda01       1981          12        27        Denver, CO   
1      aaronha01       1934           2         5        Mobile, AL   
2      aaronto01       1939           8         5        Mobile, AL   
3       aasedo01       1954           9         8        Orange, CA   
4       abadan01       1972           8        25    Palm Beach, FL   
...          ...        ...         ...       ...               ...   
20751  thomacl03       1896          11        25       Greenup, KY   
20752  malarda01       1894          10        18    White Hall, LA   
20753  gardnje02       1895           9        27  Russellville, AK   
20754   pagete01       1903           4        22       Glasgow, KY   
20755  mcdonwe01       1900           1         1       Glasgow, DE   

      birthCountry birthState nameFirst   nameLast  weight  ...       debut  \
0              USA         CO     David    Aardsma     215  ...  200